<a href="https://colab.research.google.com/github/dnguyend/rayleigh_newton/blob/master/colab/UZPairsEigenTensor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

$\newcommand{\cT}{\mathcal{T}}$
$\newcommand{\cB}{\mathcal{B}}$
$\newcommand{\C}{\mathbb{C}}$
$\newcommand{\R}{\mathbb{R}}$
# This workbook compute all complex eigentensor unitary Z-pairs by Rayleigh quotient iteration, an extension of real Z pairs
 Codes and examples for the paper "Rayleigh Quotient Iteration and convergence analysis of feasibility perturbed higher-order constrained iterations"
 Section 6.1

The problem is
$$\cT(X^{[m-1]}) -X\lambda = 0 \\
X^*X = 1$$
here, $\cT(X^{[m-1]}) = \cT(I, X^{[m-1]})$ is a vector-valued function from $\C^n$ to itself, each entry is a homogeneous polynomial of order $m-1$. $\cT$ could be represented by a tensor (a multi-linear function). The square bracket means the number of times $X$ is repeated. $X^*$ is the hermitian transpose.
* $\cT$ and is a real $(m, n)$ tensors
* $X\in \C^n, \lambda\in \R$. 
* The number of eigenvalues is $\sum_{i=0}^{n-1}(m-1)^i$, or $\frac{(m-1)^n-1}{m-2}$ if $m\neq 2$.
* An extension of the Rayleigh quotient iteration. We also implement the real version, including a Rayleigh-Chebyshev algorithm.




* We can view the code as is without rerunning. Alternatively, we can execute cell by cell. Function definition cells does not return outputs.

* We rerun some examples from previous works, including [CARTWRIGHT, STURMFELS (2013)] [Cui et al. (2014)] for comparison. We found all eigenpairs associated with the Motzkin polynomial, including two pairs with multiplicity.

* We use symbolic calculations to show example 4.7 of [Cui et al. (2014)] has an infinite family of eigenvectors, and Newton regularity condition is not satisfied for these pairs.

CARTWRIGHT , D. & STURMFELS , B. (2013) The number of eigenvalues of a tensor. Linear Algebra and its Applications, 438, 942 – 952. Tensors and Multilinear Algebra.

CUI , C.-F., DAI , Y.-H. & NIE , J. (2014) All real eigenvalues of symmetric tensors. SIAM Journal on MatrixAnalysis and Applications, 35, 1582–1601.

# The code:
To keep the project compact we put several functions in libraries.  The module *utils* has some helper functions to generate a random symmetric tensor, and to generate a tensor from a polynomial given symbolically.

# First, clone the project from github.


In [ ]:
!git clone https://github.com/dnguyend/rayleigh_newton

Cloning into 'rayleigh_newton'...
remote: Enumerating objects: 299, done.
remote: Counting objects: 100% (299/299), done.
remote: Compressing objects: 100% (157/157), done.
remote: Total 299 (delta 159), reused 264 (delta 136), pack-reused 0
Receiving objects: 100% (299/299), 14.83 MiB | 38.44 MiB/s, done.
Resolving deltas: 100% (159/159), done.


Importing main functions to be used later - but the code to find all complex eigen pairs is in the next block.
some codes to beautify the outputs - and verify the outputs

In [ ]:
from __future__ import print_function
import numpy as np
import numpy.linalg as la
import pandas as pd
import sympy as sp
from types import SimpleNamespace
from time import process_time
import rayleigh_newton.core.utils as ut

from IPython.core.display import display
from rayleigh_newton.core import display_tools as dt
from rayleigh_newton.core import uz_eigen_tensor_solver as uz


In [ ]:
def check_eig(TT, all_eig):
    m = len(TT.shape)
    d = 2
    
    max_run = all_eig.lbd[~np.isnan(all_eig.lbd)].shape[0]
    diff0 = np.empty(max_run)
    diff1 = np.empty(max_run)
    for i in range(max_run):
        Tnew = ut.tv_mode_product(TT, all_eig.x[i, :], m-1)
        lbdnew = np.sum(all_eig.x[i, :].conjugate()*Tnew).real
        diff0[i] = lbdnew - all_eig.lbd[i]
        diff1[i] = la.norm(Tnew - lbdnew*all_eig.x[i, :])
    print('check lbd %f' % np.max(np.abs(diff0)))
    print('check equation %f' % np.max(np.abs(diff1)))

    diff3 = np.empty(max_run-1)
    for i in range(max_run-1):
        if i > 0:
            factors = np.abs(np.abs(np.sum(all_eig.x[i, :][None, :]*all_eig.x[:i, :], axis=1)) - 1)
            diff3[i] = factors[np.argmax(factors)]
        
    print("check uniqueness")
    print(np.sort(diff3[1:][np.where(diff3 < 1e-1)]))


# Example 4.2 of [Cui et. al]
CUI , C.-F., DAI , Y.-H. & NIE , J. (2014) All real eigenvalues of symmetric tensors. SIAM Journal on MatrixAnalysis and Applications, 35, 1582–1601.

According to the paper, it took 400 seconds (in a Windows machine around 10 years ago) to find these real eigenpairs.

It takes $2$ to $4$ seconds on colab using our algorithm to find all 85 complex eigen pairs

In [ ]:
n = 4
m = 5

def rand4():
    X = np.random.randn(4)
    return X/la.norm(X)

w10, w11, w12, w13 = rand4()
w20, w21, w22, w23 = rand4()
w30, w31, w32, w33 = rand4()
x0, x1, x2, x3 = sp.symbols('x0 x1 x2 x3')
w1 = sp.Matrix([[w10, w11, w12, w13]]).T
w2 = sp.Matrix([[w20, w21, w22, w23]]).T
w3 = sp.Matrix([[w30, w31, w32, w33]]).T
P = (sp.eye(4) - 2*w1*w1.T)*(sp.eye(4) - 2*w2*w2.T)*(sp.eye(4) - 2*w3*w3.T)
XX = sp.Matrix([[x0, x1, x2, x3]]).T
PX = P*XX

AP = sp.expand(PX[0]**m + 2*PX[1]**m - 3*PX[2]**m - 4*PX[3]**m)
TT = ut.generate_symmetric_tensor_from_poly(XX, AP)
X = np.random.randn(n)
print("verifying that the polynomial function and the tensor evaluates to the same number")
print(AP.subs([(XX[i], X[i]) for i in range(n)]))
print(ut.tv_mode_product(TT, X, m))
    

verifying that the polynomial function and the tensor evaluates to the same number
-494.480511953080
-494.48051195307994


Now run the code. The first table shows the eigenvalue and the vector next to it. The next one shows complex pairs. To analyze further the user can examine all_eig, which has the fields listed below

In [ ]:

n_eig = uz.complex_eigen_cnt(m, n)
print("expecting  %d eigenvalues" % n_eig)
all_eig = SimpleNamespace(
    lbd=np.full((n_eig), np.nan, dtype=complex),
    x=np.full((n_eig, n), np.nan, dtype=complex),
    is_self_conj=np.zeros((n_eig), dtype=bool),
    is_real=np.zeros((n_eig), dtype=bool))
_, all_eig = uz.time_find_all_unitary(TT, tol=1e-6, disc=1e-3, max_itr=200)
display(dt.display_all_real(all_eig).sort_values(by='lbd', ascending=False))
display(dt.display_all_complex(all_eig))
check_eig(TT, all_eig)

expecting  85 eigenvalues
Found 5 eigenpairs
Found 14 eigenpairs
Found 15 eigenpairs
Found 25 eigenpairs
Found 35 eigenpairs
Found 45 eigenpairs
Found 55 eigenpairs
Found 60 eigenpairs
Found 65 eigenpairs
Found 70 eigenpairs
Found 75 eigenpairs
Found 85 eigenpairs
tot time 2.968194 avg=0.034920


,lbd,0,1,2,3
65,4.000000,-0.556822,0.672663,-0.031851,0.486271
60,3.000000,-0.554909,-0.735501,0.051144,0.385355
13,2.000000,-0.540562,0.018380,-0.498991,-0.677099
46,1.216340,-0.785147,-0.092033,0.016435,0.612212
17,1.000000,0.299696,-0.078904,-0.864510,0.395700
66,0.961091,-0.769572,0.432579,-0.410646,-0.228046
59,0.854277,-0.772175,-0.470043,-0.342146,-0.256403
0,0.605725,-0.043219,0.291778,-0.748445,0.593993
45,0.554980,-0.069898,-0.483927,-0.681302,0.544754
72,0.540241,-0.948482,-0.058337,-0.310022,0.029408


,lbd,0,1,2,3
3,0.346561+0.000000j,-0.653012-0.182308j,0.335608+0.047998j,0.011338+0.525889j,-0.301288-0.240708j
4,0.346561+0.000000j,-0.653012+0.182308j,0.335608-0.047998j,0.011338-0.525889j,-0.301288+0.240708j
5,0.251839+0.000000j,0.556973-0.167615j,-0.027211-0.055130j,-0.425748+0.225018j,0.238484+0.607567j
6,0.251839+0.000000j,0.556973+0.167615j,-0.027211+0.055130j,-0.425748-0.225018j,0.238484-0.607567j
7,0.251839+0.000000j,0.273085+0.052158j,0.047530-0.475474j,0.393161+0.719762j,-0.136344+0.056097j
...,...,...,...,...,...
80,0.854277+0.000000j,-0.224563+0.316164j,0.255785+0.419057j,-0.392617-0.029139j,-0.636690-0.219559j
81,0.251839+0.000000j,-0.133328+0.238355j,0.061350+0.090304j,0.018003+0.464377j,-0.645410-0.530064j
82,0.251839+0.000000j,-0.133328-0.238355j,0.061350-0.090304j,0.018003-0.464377j,-0.645410+0.530064j
83,1.216340+0.000000j,-0.223489+0.324274j,-0.770539-0.391736j,0.048562+0.018549j,0.121717-0.283188j


check lbd 0.000000
check equation 0.000000
check uniqueness
[]


# Example 4.3 from Cui, Dai et. al
$T = 2x_0^4 + 3x_1^4 + 5x_2^4 + 4ax_0^2x_1x_2$

It finds all 13 pairs instantly, less than a second. Setting up the tensor then run the eigenpair timing code. Take $a=3$ below, but we can just change $a$ then rerun

In [ ]:
n = 3
m = 4
x0, x1, x2 = sp.symbols('x0 x1 x2')
XX = sp.Matrix([x0, x1, x2])
a = 3
P = 2*x0**4 + 3*x1**4 + 5*x2**4 + 4*a*x0**2*x1*x2
T = ut.generate_symmetric_tensor_from_poly(XX, P)
X = np.random.randn(3)
print(P.subs([(XX[i], X[i]) for i in range(3)]))
print(ut.tv_mode_product(T, X, 4))


65.3083856871975
65.30838568719751


In [ ]:
n_eig = uz.complex_eigen_cnt(m, n)
print("expecting  %d eigenvalues" % n_eig)
all_eig = SimpleNamespace(
    lbd=np.full((n_eig), np.nan, dtype=complex),
    x=np.full((n_eig, n), np.nan, dtype=complex),
    is_self_conj=np.zeros((n_eig), dtype=bool),
    is_real=np.zeros((n_eig), dtype=bool))
_, all_eig = uz.time_find_all_unitary(T, tol=1e-6, disc=1e-3, max_itr=200)

display(dt.display_all_real(all_eig).sort_values(by='lbd', ascending=False))
check_eig(T, all_eig)

expecting  13 eigenvalues
Found 5 eigenpairs
Found 7 eigenpairs
Found 10 eigenpairs
Found 13 eigenpairs
tot time 0.300042 avg=0.023080


,lbd,0,1,2
10,5.000000,-4.659959e-25,3.257428e-25,-1.000000e+00
11,3.000000,-1.253226e-14,-1.000000e+00,-1.958818e-14
3,2.214715,4.875358e-01,-7.928607e-01,-3.656237e-01
6,2.214715,-4.875358e-01,-7.928607e-01,-3.656237e-01
7,2.000000,1.000000e+00,-1.382570e-20,-1.381195e-20
4,1.875000,1.377351e-26,-7.905694e-01,6.123724e-01
5,1.875000,-1.101114e-16,-7.905694e-01,-6.123724e-01
0,-0.512644,-7.042264e-01,-5.266924e-01,4.760885e-01
12,-0.512644,-7.042264e-01,5.266924e-01,-4.760885e-01


check lbd 0.000000
check equation 0.000000
check uniqueness
[]


# Example 4.6 from [Cui et al.], or 9.1 from [Qi et al.]
[L. Qi, F. Wang, and Y. Wang.] Z-eigenvalue methods for a global polynomial optimization problem, Mathematical Programming, 118 (2009), pp. 301–316.

There is a typo in [Cui et al.]. According to [Qi et al.], the polynomial is
$\sum_{i=0}^5(i+1)x_i^3 + 30\sum_{i=0}^4x_i^2x_{i+1} $
Numerically, the result confirm the coefficients in [Qi, Wang et al.]. It takes $3$ to $4$ seconds in our algorithm, and took 10870 seconds in the algorithm in Cui, Dai et Al.

In [ ]:
n = 6
m = 3
x0, x1, x2, x3, x4, x5 = sp.symbols('x0 x1 x2 x3 x4 x5')
XX = sp.Matrix([x0, x1, x2, x3, x4, x5])

P = 0
for i in range(n):
    P = P + (i+1)*XX[i]**3
for i in range(n-1):
    P = P + 30*XX[i]**2*XX[i+1]
T = ut.generate_symmetric_tensor_from_poly(XX, P)
X = np.random.randn(n)
print(P.subs([(XX[i], X[i]) for i in range(n)]))
print(ut.tv_mode_product(T, X, m))

n_eig = uz.complex_eigen_cnt(m, n)
print("expecting  %d eigenvalues" % n_eig)
all_eig = SimpleNamespace(
    lbd=np.full((n_eig), np.nan, dtype=complex),
    x=np.full((n_eig, n), np.nan, dtype=complex),
    is_self_conj=np.zeros((n_eig), dtype=bool),
    is_real=np.zeros((n_eig), dtype=bool))
_, all_eig = uz.time_find_all_unitary(T, tol=1e-6, disc=1e-3, max_itr=200)
display(dt.display_all_real(all_eig).sort_values(by='lbd', ascending=False))
check_eig(T, all_eig)

-155.829152769413
-155.82915276941338
expecting  63 eigenvalues
Found 5 eigenpairs
Found 15 eigenpairs
Found 15 eigenpairs
Found 20 eigenpairs
Found 25 eigenpairs
Found 30 eigenpairs
Found 35 eigenpairs
Found 45 eigenpairs
Found 55 eigenpairs
Found 63 eigenpairs
tot time 5.867503 avg=0.093135


,lbd,0,1,2,3,4,5
16,16.234514,-3.294437e-82,-3.863486e-26,-8.941490e-21,6.576900e-01,6.801877e-01,0.323711
52,15.455152,9.183550e-41,-4.708899e-32,-4.635987e-21,-2.059282e-01,8.139433e-01,0.543222
42,15.429818,2.465190e-31,2.358300e-27,-8.588582e-23,1.352303e-20,8.249106e-01,0.565263
30,10.971097,-2.070309e-14,-5.816987e-13,-9.094898e-13,-4.285536e-14,-6.922834e-01,0.721626
62,8.734734,3.666544e-01,4.184040e-01,2.342437e-01,2.792536e-02,-5.512904e-01,0.575267
8,8.659601,-2.973235e-01,4.478462e-01,2.894994e-01,4.315336e-02,-5.467213e-01,0.571363
7,8.597884,-9.161187e-24,4.759539e-01,3.822988e-01,7.627314e-02,-5.434459e-01,0.571108
51,8.188851,1.619654e-16,-4.543818e-01,4.548807e-01,1.142687e-01,-5.188059e-01,0.551728
31,7.216542,4.882911e-20,-6.304747e-19,6.395782e-01,2.648904e-01,-4.642822e-01,0.552463
28,6.000000,1.441391e-25,-1.853680e-22,-1.974054e-22,-3.899110e-22,1.058937e-23,1.000000


check lbd 0.000000
check equation 0.000000
check uniqueness
[]


# Example 4.7 of [Cui, Dai et al.] 

There are 364 eigenpairs, it takes a long time to find them, as this is a degenerate case. The eigenvalue $-\frac{9}{2}$ has infinite number of eigenvectors, and we show the regularity condition required by Newton method fails.

In [ ]:
import sympy as sp
x0, x1, x2, x3, x4, x5 = sp.symbols('x0 x1 x2 x3 x4 x5')
XX = sp.Matrix([x0, x1, x2, x3, x4, x5])
P = 0
for i in range(5):
    for j in range(i, 6):
        P = P - (XX[i]-XX[j])**4
TT = ut.generate_symmetric_tensor_from_poly(XX, P)

X = np.random.randn(6)
print(P.subs([(XX[i], X[i]) for i in range(6)]))
print(ut.tv_mode_product(TT, X, 4))

n = 6
m = 4
np.random.seed(1)
n_eig = uz.complex_eigen_cnt(m, n)
print("expecting  %d eigenvalues" % n_eig)
n_eig += 10
all_eig = SimpleNamespace(
    lbd=np.full((n_eig), np.nan, dtype=complex),
    x=np.full((n_eig, n), np.nan, dtype=complex),
    is_self_conj=np.zeros((n_eig), dtype=bool),
    is_real=np.zeros((n_eig), dtype=bool))
_, all_eig = uz.time_find_all_unitary(TT, tol=1e-7, disc=5e-3, max_itr=200)

-388.652587645519
-388.652587645519
expecting  364 eigenvalues
Found 1 eigenpairs
Found 5 eigenpairs
Found 10 eigenpairs
Found 15 eigenpairs
Found 20 eigenpairs
Found 35 eigenpairs
Found 40 eigenpairs
Found 50 eigenpairs
Found 55 eigenpairs
Found 60 eigenpairs
Found 65 eigenpairs
Found 70 eigenpairs
Found 75 eigenpairs
Found 80 eigenpairs
Found 90 eigenpairs
Found 95 eigenpairs
Found 100 eigenpairs
Found 110 eigenpairs
Found 120 eigenpairs
Found 130 eigenpairs
Found 140 eigenpairs
Found 150 eigenpairs
Found 160 eigenpairs
Found 170 eigenpairs
Found 175 eigenpairs
Found 180 eigenpairs
Found 190 eigenpairs
Found 200 eigenpairs
Found 210 eigenpairs
Found 215 eigenpairs
Singular matrix
Found 225 eigenpairs
Singular matrix
Singular matrix
Found 235 eigenpairs
Found 240 eigenpairs
Found 245 eigenpairs
Found 255 eigenpairs
Found 260 eigenpairs
Found 270 eigenpairs
Found 280 eigenpairs
Singular matrix
Found 290 eigenpairs
Found 295 eigenpairs
Found 300 eigenpairs
Singular matrix
Found 320 eige

In [ ]:
check_eig(TT, all_eig)
display(dt.display_all_real(all_eig).sort_values(by='lbd'))


check lbd 0.000000
check equation 0.000000
check uniqueness
[1.]


,lbd,0,1,2,3,4,5
36,-7.2,-1.825742e-01,-1.825742e-01,-1.825742e-01,-1.825742e-01,-1.825742e-01,9.128709e-01
101,-7.2,9.128709e-01,-1.825742e-01,-1.825742e-01,-1.825742e-01,-1.825742e-01,-1.825742e-01
120,-7.2,-1.825742e-01,-1.825742e-01,9.128709e-01,-1.825742e-01,-1.825742e-01,-1.825742e-01
7,-7.2,-1.825742e-01,-1.825742e-01,-1.825742e-01,9.128709e-01,-1.825742e-01,-1.825742e-01
42,-7.2,-1.825742e-01,9.128709e-01,-1.825742e-01,-1.825742e-01,-1.825742e-01,-1.825742e-01
163,-7.2,-1.825742e-01,-1.825742e-01,-1.825742e-01,-1.825742e-01,9.128709e-01,-1.825742e-01
146,-6.0,7.071068e-01,4.645133e-16,-7.071068e-01,4.288319e-16,4.080255e-16,3.197902e-16
82,-6.0,-7.071068e-01,-3.545818e-15,-3.665601e-15,-3.576547e-15,-3.814350e-15,7.071068e-01
74,-6.0,-1.557873e-15,-7.071068e-01,7.071068e-01,-7.435155e-16,-7.694514e-16,2.379930e-16
38,-6.0,-1.380359e-16,-8.003337e-17,7.071068e-01,-1.420605e-16,-7.071068e-01,1.804253e-17


We show $\lambda = -9/2$ has infinitely many eigenvectors of the form
$$\begin{bmatrix}x_0\\x_0\\x_2\\x_2\\-x_0-x_2\\-x_0-x_2\end{bmatrix}$$
with $x_2 =\frac{-x_0 \pm \sqrt{1-3x_0^2}}{2}$
We got this solution by inspecting the eigenvectors of $\lambda=-\frac{9}{2}$ found numerically above

In [ ]:
x0, x1, x2, x3, x4, x5, z = sp.symbols('x0 x1 x2 x3 x4 x5 z')
XX = sp.Matrix([x0, x1, x2, x3, x4, x5])
P = 0
for i in range(5):
  for j in range(i, 6):
    P = P - (XX[i]-XX[j])**4
gr = sp.Matrix([sp.diff(P, XX[i]) for i in range(6)])/sp.Integer(4)
grs = gr.subs([(x1, x0), (x3, x2), (x5, x4)])

grs2a = sp.expand(grs.subs(x4, -x0  -x2))
x2s = (-x0 - sp.sqrt(1-3*x0*x0))/2
Xnew = sp.Matrix([x0, x0, x2s, x2s, -x0-x2s, -x0-x2s])


print("CHECK NORM IS 1 and eigenvector")
grs2b = sp.simplify(sp.expand(grs2a.subs(x2, x2s)))
display(sp.simplify(Xnew.dot(Xnew)))

display(sp.simplify(grs2b + sp.Integer(9)/sp.Integer(2)* Xnew))



CHECK NORM IS 1 and eigenvector


1

Matrix([
[0],
[0],
[0],
[0],
[0],
[0]])

An orthogonal complement of Xnew using symbolic Householder

In [ ]:
n = 6
m = 4

I2 = sp.eye(n)[:, 1:]
v = Xnew  - sp.Matrix([1] + (n-1)*[0])
v = v / sp.sqrt(v.dot(v))
Q = I2 - 2*v.reshape(n, 1)*v.reshape(1, n)*I2
display(Q)
display(sp.simplify(sp.expand(Xnew.T*Q)))
display(sp.simplify(sp.expand(Q.T*Q)))



Matrix([
[                     -2*x0*(x0 - 1)/(x0**2 + 2*(-x0/2 - sqrt(1 - 3*x0**2)/2)**2 + 2*(-x0/2 + sqrt(1 - 3*x0**2)/2)**2 + (x0 - 1)**2),                      -2*(-x0/2 - sqrt(1 - 3*x0**2)/2)*(x0 - 1)/(x0**2 + 2*(-x0/2 - sqrt(1 - 3*x0**2)/2)**2 + 2*(-x0/2 + sqrt(1 - 3*x0**2)/2)**2 + (x0 - 1)**2),                      -2*(-x0/2 - sqrt(1 - 3*x0**2)/2)*(x0 - 1)/(x0**2 + 2*(-x0/2 - sqrt(1 - 3*x0**2)/2)**2 + 2*(-x0/2 + sqrt(1 - 3*x0**2)/2)**2 + (x0 - 1)**2),                      -2*(-x0/2 + sqrt(1 - 3*x0**2)/2)*(x0 - 1)/(x0**2 + 2*(-x0/2 - sqrt(1 - 3*x0**2)/2)**2 + 2*(-x0/2 + sqrt(1 - 3*x0**2)/2)**2 + (x0 - 1)**2),                      -2*(-x0/2 + sqrt(1 - 3*x0**2)/2)*(x0 - 1)/(x0**2 + 2*(-x0/2 - sqrt(1 - 3*x0**2)/2)**2 + 2*(-x0/2 + sqrt(1 - 3*x0**2)/2)**2 + (x0 - 1)**2)],
[                       -2*x0**2/(x0**2 + 2*(-x0/2 - sqrt(1 - 3*x0**2)/2)**2 + 2*(-x0/2 + sqrt(1 - 3*x0**2)/2)**2 + (x0 - 1)**2) + 1,                            -2*x0*(-x0/2 - sqrt(1 - 3*x0**2)/2)/(x0**2 + 2*(-x0/2 -

Matrix([[0, 0, 0, 0, 0]])

Matrix([
[1, 0, 0, 0, 0],
[0, 1, 0, 0, 0],
[0, 0, 1, 0, 0],
[0, 0, 0, 1, 0],
[0, 0, 0, 0, 1]])

Show the $F^{\natural}$ does not exists  at these eigenpairs by showing $Q^T((m-1)\cT(I, I, X^{[m-2]}) - \lambda I)Q$ is not invertible.

In [ ]:
hess = sp.zeros(6, 6)
for i in range(6):
  hess[:, i] = sp.diff(gr, XX[i])
hessp = Q.T*(hess - P*sp.eye(n)).subs([(XX[i], Xnew[i]) for i in range(6)])*Q
sp.simplify(hessp.det())

0

In [ ]:
dt.display_all_complex(all_eig)

,lbd,0,1,2,3,4,5
2,4.485401+0.000000j,-0.018377+0.353460j,0.021527+0.218105j,0.021527+0.218105j,-0.003150-0.571564j,-0.018377+0.353460j,-0.003150-0.571564j
3,4.485401+0.000000j,-0.018377-0.353460j,0.021527-0.218105j,0.021527-0.218105j,-0.003150+0.571564j,-0.018377-0.353460j,-0.003150+0.571564j
4,4.499816+0.000000j,-0.001675-0.442877j,0.002572-0.099325j,-0.000897+0.542203j,0.002572-0.099325j,-0.000897+0.542203j,-0.001675-0.442877j
8,4.499189+0.000000j,-0.005262-0.161062j,0.001307+0.560658j,0.001307+0.560658j,0.003955-0.399596j,-0.005262-0.161062j,0.003955-0.399596j
10,3.976159+0.000000j,0.005401-0.559875j,-0.123238+0.261123j,0.117837+0.298752j,0.117837+0.298752j,-0.123238+0.261123j,0.005401-0.559875j
...,...,...,...,...,...,...,...
359,4.297238+0.000000j,0.054244-0.445152j,0.054244-0.445152j,0.031405+0.532007j,-0.085649-0.086854j,0.031405+0.532007j,-0.085649-0.086854j
360,4.472286+0.000000j,0.009581-0.550083j,0.021686+0.424328j,-0.031267+0.125756j,0.021686+0.424328j,0.009581-0.550083j,-0.031267+0.125756j
361,4.472286+0.000000j,0.009581+0.550083j,0.021686-0.424328j,-0.031267-0.125756j,0.021686-0.424328j,0.009581+0.550083j,-0.031267-0.125756j
362,4.493796+0.000000j,0.011497+0.376163j,0.002808-0.567164j,0.002808-0.567164j,0.011497+0.376163j,-0.014304+0.191001j,-0.014304+0.191001j


# Motzkin polynomial CARTWRIGHT , D. & STURMFELS , B. (2013), example 5.9

The paper convention using a tensor 6 times our tensor. So eigenvalues there dividing by 6 will match ours.

In this case, we find only 23 eigenpairs.
* The "missing" 8 are actually eigenpairs with multiplicity. A small perturbation shows there are 14 eigenvectors with eigenvalue 0, but collapse to only 6.
* Comparing with Cartwright- Stumfels 2013, counting only 25 pairs, their "missing pairs" are 6 complex pairs of eigenvalues 
  * 1/12 complex eigen vectors $(x_0, -\bar{x_0}, 0)$ with $x_0^4=-\frac{1}{4}$ (2 equivalent pairs)
  * 3/16, eigenvectors $(0, 1, 0), (1, 0, 0), (x_0, x_1, \frac{\sqrt{2}}{2})$, $x_0^2 = x_1^2 =-\frac{1}{4}$ (4 equivalent pairs)
* The cited paper uses the normalization $X^TX = 1$, these vectors satisfy $X^TX =0$.

Gradient:
$$6T(I, X^{[5]}) = \left[\begin{matrix}4 x_{0}^{3} x_{1}^{2} + 2 x_{0} x_{1}^{4} - 6 x_{0} x_{1}^{2} x_{2}^{2}\\2 x_{0}^{4} x_{1} + 4 x_{0}^{2} x_{1}^{3} - 6 x_{0}^{2} x_{1} x_{2}^{2}\\- 6 x_{0}^{2} x_{1}^{2} x_{2} + 6 x_{2}^{5}\end{matrix}\right]$$

CARTWRIGHT , D. & STURMFELS , B. (2013) The number of eigenvalues of a tensor. Linear Algebra and its
Applications, 438, 942 – 952. Tensors and Multilinear Algebra.

In [ ]:
n = 3
m = 6
x0, x1, x2 = sp.symbols('x0 x1 x2')
XX = sp.Matrix([x0, x1, x2])

P = x0**4*x1**2 + x0**2*x1**4 + x2**6 - 3*x0**2*x1**2*x2**2
T = ut.generate_symmetric_tensor_from_poly(XX, P)
X = np.random.randn(3)
print(P.subs([(XX[i], X[i]) for i in range(n)]))
print(uz.tv_mode_product(T, X, m))

np.random.seed(0)
n_eig = uz.complex_eigen_cnt(n, m)
print("expecting  %d eigenvalues" % n_eig)
all_eig = SimpleNamespace(
    lbd=np.full((n_eig), np.nan, dtype=complex),
    x=np.full((n_eig, n), np.nan, dtype=complex),
    is_self_conj=np.zeros((n_eig), dtype=bool),
    is_real=np.zeros((n_eig), dtype=bool))

_, all_eig = uz.time_find_all_unitary(
    T, tol=1e-6, disc=5e-3, max_itr=200, max_test=2e3)

0.0744708260115966
0.07447082601159666
expecting  63 eigenvalues
Found 5 eigenpairs
Found 5 eigenpairs
Found 10 eigenpairs
Singular matrix
Singular matrix
Found 20 eigenpairs


/content/rayleigh_newton/core/uz_eigen_tensor_solver.py:373: RuntimeWarning: divide by zero encountered in double_scalars
  np.sum((x_k.conjugate() * lhs[:, 0]).real)) - lhs[:, 1]
/content/rayleigh_newton/core/uz_eigen_tensor_solver.py:387: RuntimeWarning: invalid value encountered in true_divide
  x_k_n = (x_k + y) / norm(x_k + y)


Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular

/content/rayleigh_newton/core/uz_eigen_tensor_solver.py:373: RuntimeWarning: invalid value encountered in double_scalars
  np.sum((x_k.conjugate() * lhs[:, 0]).real)) - lhs[:, 1]


Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Found 23 eigenpairs
tot time 48.939865 avg=1.578705


In [ ]:
check_eig(T, all_eig)
display(dt.display_all_real(all_eig).sort_values(by='lbd'))

check lbd 0.000000
check equation 0.000000
check uniqueness
[]


,lbd,0,1,2
3,-1.347084e-17,5.773503e-01,-5.773503e-01,-5.773503e-01
1,-4.641582e-34,-3.847635e-295,-1.000000e+00,2.308716e-06
2,2.818596e-33,1.000000e+00,1.056484e-272,3.758421e-06
19,8.595497e-21,-5.773503e-01,-5.773503e-01,5.773503e-01
4,2.003086e-18,-5.773503e-01,5.773503e-01,-5.773503e-01
22,4.006172e-18,-5.773503e-01,-5.773503e-01,-5.773503e-01
16,1.562500e-02,-8.253401e-01,2.623238e-01,5.000000e-01
11,1.562500e-02,-2.623238e-01,8.253401e-01,-5.000000e-01
0,1.562500e-02,8.253401e-01,2.623238e-01,5.000000e-01
7,1.562500e-02,2.623238e-01,8.253401e-01,-5.000000e-01


In [ ]:
dt.display_all_complex(all_eig)

,lbd,0,1,2
5,0.083333+0.000000j,-0.5+0.5j,0.5+0.5j,-5.591232e-22+3.705890e-22j
6,0.083333+0.000000j,-0.5-0.5j,0.5-0.5j,-5.591232e-22-3.705890e-22j
14,0.187500+0.000000j,-0.0-0.5j,0.0+0.5j,-7.071068e-01+5.551115e-17j
15,0.187500+0.000000j,-0.0+0.5j,0.0-0.5j,-7.071068e-01-5.551115e-17j
17,0.187500+0.000000j,-0.5-0.0j,-0.5-0.0j,6.555831e-17+7.071068e-01j
18,0.187500+0.000000j,-0.5+0.0j,-0.5+0.0j,6.555831e-17-7.071068e-01j
23,N00000000a00000000N,N000a000N,N000a000N,N000000000000a000000000000N
24,N00000000a00000000N,N000a000N,N000a000N,N000000000000a000000000000N
25,N00000000a00000000N,N000a000N,N000a000N,N000000000000a000000000000N
26,N00000000a00000000N,N000a000N,N000a000N,N000000000000a000000000000N


perturb, we see the there are actually 14 eigenvector of eigenvalue zero, they just collapse to 6 above

In [ ]:
Ta = T.copy()
for i in range(n):
  Ta[m*[i]] += (i+1)*1e-5

all_eig2 = SimpleNamespace(
    lbd=np.full((n_eig), np.nan, dtype=complex),
    x=np.full((n_eig, n), np.nan, dtype=complex),
    is_self_conj=np.zeros((n_eig), dtype=bool),
    is_real=np.zeros((n_eig), dtype=bool))

_, all_eig2 = uz.time_find_all_unitary(Ta, tol=1e-6, disc=5e-3, max_itr=200, max_test=2e3)
check_eig(Ta, all_eig2)
dt.display_all_real(all_eig2).sort_values(by='lbd')

Found 5 eigenpairs
Found 13 eigenpairs
Found 15 eigenpairs
Found 20 eigenpairs
Found 25 eigenpairs
Found 30 eigenpairs
Found 31 eigenpairs
tot time 0.842076 avg=0.027164
check lbd 0.000001
check equation 0.000000
check uniqueness
[0.02680511 0.03080264 0.03080264 0.78745263]


,lbd,0,1,2
15,-7.406198e-07,0.577338,-0.577364,5.773480e-01
27,-2.221975e-11,-0.577353,-0.577344,5.773532e-01
19,1.481183e-06,0.577364,-0.577338,-5.773487e-01
2,5.402014e-06,0.993842,-0.000034,-1.108045e-01
29,1.075728e-05,0.000017,-0.993764,1.115013e-01
18,5.404407e-04,-0.578528,-0.576406,-5.771151e-01
12,1.562489e-02,-0.825338,-0.262321,5.000055e-01
26,1.562500e-02,0.825340,-0.262324,-5.000000e-01
17,1.562500e-02,0.262324,-0.825340,5.000000e-01
5,1.562529e-02,0.262312,0.825343,-5.000012e-01


And 8 eigenvectors of small eigenvalues in the complex part

In [ ]:
dt.display_all_complex(all_eig2)


,lbd,0,1,2
0,0.000016+0.000000j,-0.985789-0.098879j,0.000094-0.000035j,-0.111919+0.076924j
1,0.000016+0.000000j,-0.985789+0.098879j,0.000094+0.000035j,-0.111919-0.076924j
3,0.000008+0.000000j,0.982799-0.141389j,-0.000043-0.000018j,-0.032650+0.114231j
4,0.000008+0.000000j,0.982799+0.141389j,-0.000043+0.000018j,-0.032650-0.114231j
7,0.000016+0.000000j,-0.000009-0.000021j,-0.142265+0.982614j,0.114927-0.031958j
8,0.000016+0.000000j,-0.000009+0.000021j,-0.142265-0.982614j,0.114927+0.031958j
9,0.000032+0.000000j,-0.000017-0.000047j,-0.097297+0.986077j,0.075716+0.111587j
10,0.000032+0.000000j,-0.000017+0.000047j,-0.097297-0.986077j,0.075716-0.111587j
13,0.083348+0.000000j,-0.500022-0.499992j,0.500022-0.499962j,-0.000000-0.000090j
14,0.083348+0.000000j,-0.500022+0.499992j,0.500022+0.499962j,-0.000000+0.000090j


# Example 4.11
In these examples, we show the operator $\Pi L_{X}$ is not invertible, a small perturbation gives us information on other eigenpairs

In [43]:
n = 5
m = 3
np.random.seed(0)
neig0 = uz.complex_eigen_cnt(n, m)
T = np.zeros((n, n, n))

def mpw(i):
  return 1 - 2*(i % 2)
mpw(6)

for i0 in range(n):
  for i1 in range(n):
    for i2 in range(n):
          T[i0, i1, i2] = mpw(i0+1)/(i0+1) + mpw(i1+1)/(i1+1) + mpw(i2+1)/(i2+1)

sch_x, sch_lbd, ctr, converge, err = uz.schur_form_rayleigh_chebyshev(
              T, max_itr=30, delta=1e-8, x_init=np.array([1] + (n-1)*[0.]), do_chebyshev=False)

print(sch_x, sch_lbd, ctr, converge, err)

from scipy.linalg import null_space
Q = null_space(sch_x.reshape(1, -1))
hess = Q.T@((m-1)*ut.tv_mode_product(T, sch_x, m-2))@Q
print(np.linalg.det(hess))


[ 0.10938332 -0.44766753 -0.24032136  0.81962463 -0.24101905] -1.8674273216398812e-17 30 False 8.2401295180647e-17
4.127827387189961e-52


A heuristic fix: Add a small tensor

In [44]:
eps = 1e-2
print("expecting %d pairs" % uz.complex_eigen_cnt(m, n))
Ta = T.copy()
for i0 in range(n):          
  # T[i0, i0, i0] += eps*np.random.uniform(1e-1, 1)
  Ta[i0, i0, i0] += eps*(i0+1)
ret = uz.time_find_all_unitary(Ta, tol=1e-6, disc=1e-3, max_itr=200)
check_eig(Ta, ret[1])
display(dt.display_all_real(ret[1]).sort_values(by='lbd', ascending=False))
dt.display_all_complex(ret[1])


expecting 31 pairs
Found 5 eigenpairs
Found 10 eigenpairs
Found 10 eigenpairs
Found 15 eigenpairs
Found 25 eigenpairs
Found 30 eigenpairs
Found 31 eigenpairs
tot time 9.841280 avg=0.317461
check lbd 0.000000
check equation 0.000000
check uniqueness
[0.77166506]


,lbd,0,1,2,3,4
29,9.966778,-0.731548,-0.137681,-0.467265,-0.236555,-0.414229
30,4.303718,-0.185353,0.715551,0.214267,0.566386,0.294833
13,0.033458,-0.317038,-0.189530,-0.236465,-0.190360,0.878301
12,0.027383,-0.250550,-0.231072,-0.219282,0.892224,-0.199203
14,0.020391,-0.262868,-0.223804,0.896644,-0.199123,-0.192853
2,0.020190,-0.332277,-0.157934,0.895570,-0.163643,-0.189274
3,0.018411,-0.403630,-0.149119,0.862896,-0.166317,-0.206387
4,0.012978,-0.276266,0.892032,-0.221386,-0.204917,-0.192234
21,0.011807,-0.424083,-0.354463,-0.317011,0.583124,0.503963
9,0.008331,-0.548293,-0.286654,0.607100,-0.251380,0.430630


,lbd,0,1,2,3,4
0,0.019433+0.000000j,0.075815-0.031845j,-0.464472+0.108510j,-0.200491+0.048191j,0.818553-0.072575j,-0.213702+0.047388j
1,0.019433+0.000000j,0.075815+0.031845j,-0.464472-0.108510j,-0.200491-0.048191j,0.818553+0.072575j,-0.213702-0.047388j
5,0.009633+0.000000j,-0.452905+0.023282j,-0.333643-0.025976j,0.634754+0.000827j,-0.272582-0.013515j,0.452698+0.003235j
6,0.009633+0.000000j,-0.452905-0.023282j,-0.333643+0.025976j,0.634754-0.000827j,-0.272582+0.013515j,0.452698-0.003235j
7,0.001508+0.000000j,0.227067-0.340693j,0.443159-0.493894j,-0.201036+0.304634j,-0.243580+0.326138j,-0.175155+0.249907j
8,0.001508+0.000000j,0.227067+0.340693j,0.443159+0.493894j,-0.201036-0.304634j,-0.243580-0.326138j,-0.175155-0.249907j
10,0.006592+0.000000j,-0.252618+0.017587j,-0.506990+0.159423j,0.533112-0.079458j,0.521758-0.102541j,-0.274333+0.069958j
11,0.006592+0.000000j,-0.252618-0.017587j,-0.506990-0.159423j,0.533112+0.079458j,0.521758+0.102541j,-0.274333-0.069958j
16,0.035229+0.000000j,-0.225050+0.038317j,-0.226601-0.050750j,-0.208713-0.003211j,-0.205097-0.031363j,0.898477+0.008943j
17,0.035229+0.000000j,-0.225050-0.038317j,-0.226601+0.050750j,-0.208713+0.003211j,-0.205097+0.031363j,0.898477-0.008943j


Now rerun the original tensor with the perturbed eigenvector found

In [45]:
x_init =np.array([-0.731364,	-0.137583,	-0.467359,	-0.236516,	-0.414504])
x_init = x_init/la.norm(x_init)
sch_x, sch_lbd, ctr, converge, err = uz.schur_form_rayleigh_chebyshev(
              T, max_itr=30, delta=1e-8, x_init=x_init, do_chebyshev=False)
print(sch_x, sch_lbd, ctr, converge, err)


[-0.73128513 -0.1375412  -0.46739894 -0.23649853 -0.4146217 ] 9.977892792857288 2 True 3.4684476073050936e-15


# Example 4.12
Error is zero, but does not converge. The run oscillates between eigenvalues. Hessian is degenerate.

In [46]:
n = 5
m = 4
np.random.seed(0)
neig0 = uz.complex_eigen_cnt(m, m)
T = np.zeros((n, n, n, n))

for i0 in range(n):
  for i1 in range(n):
    for i2 in range(n):
      for i3 in range(n):
          #T[i0, i1, i2, i3] = np.tan(i0+1) + np.tan(i1+1) + np.tan(i2+1) + np.tan(i3+1)
          T[i0, i1, i2, i3] = np.sin(i0+1 + i1+1 + i2+1 + i3+1)

sch_x, sch_lbd, ctr, converge, err = uz.schur_form_rayleigh_chebyshev(
              T, max_itr=30, delta=1e-8, x_init=np.array([1] + (n-1)*[0.]), do_chebyshev=False)


print(sch_x, sch_lbd, ctr, converge, err)

from scipy.linalg import null_space
Q = null_space(sch_x.reshape(1, -1))
hess = Q.T@((m-1)*ut.tv_mode_product(T, sch_x, m-2))@Q
print(np.linalg.det(hess))


[-0.78362815  0.21983508 -0.17942712 -0.54775296 -0.0732944 ] 3.021182773080731e-17 30 False 1.4703305038227621e-16
1.102804693282504e-51


Add a small identity matrix

In [47]:
eps = 1e-3          
print("expecting %d pairs" % uz.complex_eigen_cnt(m, n))
Ta = T.copy()
for i0 in range(n):          
  Ta[i0, i0, i0, i0] += eps*(i0+1)
ret = uz.time_find_all_unitary(Ta, tol=1e-6, disc=1e-3, max_itr=200)
check_eig(Ta, ret[1])
dt.display_all_real(ret[1]).sort_values(by='lbd', ascending=False)


expecting 121 pairs
Found 5 eigenpairs
Found 10 eigenpairs
Found 11 eigenpairs
Found 15 eigenpairs
Found 25 eigenpairs
Found 35 eigenpairs
Found 40 eigenpairs
Found 45 eigenpairs
Found 55 eigenpairs
Found 65 eigenpairs
Found 75 eigenpairs
Found 90 eigenpairs
Found 95 eigenpairs
Found 100 eigenpairs
Found 105 eigenpairs
Found 110 eigenpairs
Found 115 eigenpairs
Found 120 eigenpairs
Found 121 eigenpairs
tot time 58.064057 avg=0.479868
check lbd 0.000000
check equation 0.000000
check uniqueness
[0.7552989]


,lbd,0,1,2,3,4
119,7.260494,0.268611,0.614981,0.395899,-0.187138,-0.598239
117,4.641793,-0.505473,0.122745,0.638252,0.566929,-0.025627
99,0.002298,-0.225010,-0.456359,-0.193096,0.186404,-0.817970
94,0.002260,0.170559,0.475383,0.268212,-0.121314,0.811336
55,0.001954,0.199879,0.546671,0.233853,-0.155665,0.763073
10,0.001772,0.419511,0.088030,-0.367759,0.811174,-0.151697
114,0.001645,-0.457927,-0.130317,0.323219,-0.790438,0.209900
89,0.001478,-0.446804,-0.111437,0.433953,-0.752022,0.184651
58,0.001453,0.186391,-0.079698,-0.416384,0.688395,-0.558250
39,0.001379,0.078419,-0.152815,-0.483409,0.635507,-0.577014


Time running a random (3, 8) tensor

In [48]:
n = 8
m = 3
np.random.seed(0)
tol = 1e-6
disc = 1e-3
max_itr = 200
print("expecting %d pairs" % uz.complex_eigen_cnt(m, n))
T = ut.generate_symmetric_tensor(n, m)    
ret = uz.time_find_all_unitary(T, tol, disc, max_itr)

check_eig(T, ret[1])
dt.display_all_real(ret[1])


expecting 255 pairs
Found 16 eigenpairs
Found 20 eigenpairs
Found 25 eigenpairs
Found 35 eigenpairs
Found 55 eigenpairs
Found 65 eigenpairs
Found 75 eigenpairs
Found 85 eigenpairs
Found 95 eigenpairs
Found 105 eigenpairs
Found 110 eigenpairs
Found 115 eigenpairs
Found 125 eigenpairs
Found 130 eigenpairs
Found 135 eigenpairs
Found 140 eigenpairs
Found 145 eigenpairs
Found 160 eigenpairs
Found 175 eigenpairs
Found 185 eigenpairs
Found 195 eigenpairs
Found 205 eigenpairs
Found 210 eigenpairs
Found 215 eigenpairs
Found 225 eigenpairs
Found 235 eigenpairs


/content/rayleigh_newton/core/uz_eigen_tensor_solver.py:373: RuntimeWarning: divide by zero encountered in double_scalars
  np.sum((x_k.conjugate() * lhs[:, 0]).real)) - lhs[:, 1]
/content/rayleigh_newton/core/uz_eigen_tensor_solver.py:387: RuntimeWarning: invalid value encountered in true_divide
  x_k_n = (x_k + y) / norm(x_k + y)


Found 245 eigenpairs
Found 250 eigenpairs
Found 255 eigenpairs
tot time 137.417971 avg=0.538894
check lbd 0.000000
check equation 0.000000
check uniqueness
[0.         0.82680071]


,lbd,0,1,2,3,4,5,6,7
6,0.245118,-0.006599,-0.267517,-0.095298,0.179163,-0.017914,-0.602854,0.120986,0.713315
13,0.633309,-0.233637,-0.456184,-0.158421,0.634026,0.097317,-0.415813,0.169090,0.315058
22,0.947505,-0.211168,0.299472,0.698460,-0.069450,0.431454,-0.366871,0.030050,-0.226726
41,0.942946,-0.223762,0.357706,0.613433,-0.148033,0.457172,-0.455372,0.000741,-0.085983
42,0.950609,-0.386343,0.382425,0.535561,-0.048909,0.418312,-0.486738,-0.053171,-0.023386
43,0.652108,-0.168748,0.057854,0.505665,-0.126866,0.054132,-0.629098,0.511098,0.190972
46,0.199743,-0.386549,0.777492,0.225334,-0.015238,-0.068042,0.313337,-0.285513,0.103686
51,0.182813,-0.572902,0.642630,0.294725,0.106883,-0.127189,0.293054,-0.139914,0.197204
52,0.659164,-0.132697,-0.025948,0.072521,-0.345635,0.006218,-0.556298,0.655866,0.342533
61,0.250817,-0.368665,0.565292,0.304970,0.158155,-0.158403,0.563265,-0.265551,-0.116770


# Summarizing the result of a matlab run in a table.

We did an extensive study for various tensor size. In the display we produce the time of tweenty run for each size, the time to complete $90\%$ of the runs and to complete all the runs. Note when the number of pairs is large, the last $10\%$ takes a lot of time to complete, this is an issue with initialization rather than the speed of the RQI algorithm

In [49]:
from IPython.display import display, HTML
def check_results_complex_pairs():
    from scipy.io import loadmat
    import pandas as pd

    res = loadmat('rayleigh_newton/matlab/matlab_save_unitary_res.mat')
    n_scenarios = res['save_res'].shape[0]
    # save_mean = np.zeros((n_scenarios, 3))
    # aggregate to table:
    # (m, n, j, # eigen values, #run time 90%, run time 100%
    # then run some pandas group by statements
    sum_table = pd.DataFrame(
        {'m': np.zeros((n_scenarios), dtype=int),
         'n': np.zeros((n_scenarios), dtype=int),
         'n_trys': np.zeros((n_scenarios), dtype=int),
         'n_pairs': np.zeros((n_scenarios), dtype=int),
         'n_real_pairs': np.zeros((n_scenarios), dtype=int),
         'n_self_conj_pairs': np.zeros((n_scenarios), dtype=int),
         'n_multiple_eigen': np.zeros((n_scenarios), dtype=int),
         'time_90(s)': np.zeros((n_scenarios), dtype=float),
         'time_all(s)': np.zeros((n_scenarios), dtype=float)},
        columns=['m', 'n', 'n_trys', 'n_pairs', 'n_real_pairs',
                 'n_self_conj_pairs', 'n_multiple_eigen',
                 'time_90(s)', 'time_all(s)'])
                             
    for j in range(n_scenarios):
        m = res['save_res'][j, 0][0, 0]
        n = res['save_res'][j, 0][0, 1]
        n_trys = res['save_res'][j, 0][0, 2]

        sum_table.loc[j, 'm'] = m
        sum_table.loc[j, 'n'] = n
        sum_table.loc[j, 'n_trys'] = n_trys
        eig_cell = res['save_res'][j, 1][0]

        dtypes = eig_cell.dtype
        dtypes_dict = dict((dtypes.names[a], a)
                           for a in range(len(dtypes.names)))

        n_pairs = res['save_res'][j, 1][0][0][dtypes_dict['lbd']].shape[0]
        sum_table.loc[j, 'n_pairs'] = n_pairs

        n_real_pairs = np.sum(
            res['save_res'][j, 1][0][0][dtypes_dict['is_real']])
        sum_table.loc[j, 'n_real_pairs'] = n_real_pairs

        n_self_conj_pairs = np.sum(
            res['save_res'][j, 1][0][0][dtypes_dict['is_self_conj']])
        sum_table.loc[j, 'n_self_conj_pairs'] = n_self_conj_pairs

        # find multiple eigen:
        # typically one lbd has one or two eigen vectors.
        # some cases we have multiple eigen vectors
        # we print them out here

        u, cnt = np.unique(['%.6f' % np.abs(a) for a in
                            res['save_res'][j, 1][0][0][dtypes_dict['lbd']]],
                           return_counts=True)
        dup_cnt = [(u[aa], cnt[aa]) for aa in range(len(u)) if cnt[aa] > 2]
        if len(dup_cnt) > 0:
            print('m=%d n=%d j=%d dup=%s' % (m, n, j, str(dup_cnt)))
            sum_table.loc[j, 'n_multiple_eigen'] = np.sum(
                [a[1] for a in dup_cnt])
        sum_table.loc[j, 'time_90(s)'] = res['save_res'][j, 2][0, 0]
        sum_table.loc[j, 'time_all(s)'] = res['save_res'][j, 3][0, 0]

    # sum_by_m_n = sum_table.groupby(['m', 'n']).sum()
    mean_by_m_n = sum_table.groupby(['m', 'n']).mean()
    mean_by_m_n.n_trys = sum_table[['m', 'n', 'n_trys']].groupby(
        ['m', 'n']).count()
    mean_by_m_n.n_pairs = sum_table[['m', 'n', 'n_pairs']].groupby(
        ['m', 'n']).mean()

    mean_by_m_n.n_real_pairs = sum_table[['m', 'n', 'n_real_pairs']].groupby(
        ['m', 'n']).mean()
    mean_by_m_n.n_multiple_eigen = sum_table[['m', 'n', 'n_multiple_eigen']].groupby(
        ['m', 'n']).mean()
    # mean_by_m_n.n_multiple_eigen = mean_by_m
    # mean_by_m_n.time_90 /= mean_by_m_n.n_pairs
    # mean_by_m_n.time_all /= mean_by_m_n.n_pairs
    mean_by_m_n.drop(columns=['n_self_conj_pairs'], inplace=True)

    # from IPython.core.display import display, HTML
    with open('sum.html', 'w') as hf:
        hf.write("%s\n" % mean_by_m_n.to_html())
    with open('mean.tex', 'w') as hf:
        hf.write("%s\n" % mean_by_m_n.to_latex())
        
    with open('sum_detail.html', 'w') as hf:
        hf.write("%s\n" % sum_table.to_html())     
    display(HTML(mean_by_m_n.to_html()))
check_results_complex_pairs()        

m=3 n=9 j=142 dup=[('0.734007', 3)]
m=4 n=8 j=280 dup=[('0.212739', 4), ('0.620611', 4)]
m=4 n=8 j=283 dup=[('0.533402', 4), ('0.583325', 4)]
m=4 n=8 j=284 dup=[('0.349782', 4)]
m=4 n=8 j=288 dup=[('0.405546', 4), ('0.411682', 4), ('0.719047', 4)]
m=4 n=8 j=289 dup=[('0.390941', 4)]
m=4 n=8 j=291 dup=[('0.075326', 4), ('0.476638', 3), ('0.578372', 4)]
m=4 n=8 j=292 dup=[('0.312587', 4), ('1.325293', 4)]
m=4 n=8 j=293 dup=[('1.080718', 4), ('1.259201', 4)]
m=4 n=8 j=296 dup=[('0.345709', 3), ('0.616980', 3)]
m=4 n=8 j=297 dup=[('0.392469', 4), ('0.533838', 4)]
m=4 n=8 j=298 dup=[('0.780948', 4)]
m=4 n=8 j=299 dup=[('0.888106', 4), ('0.897943', 4)]


# Comparing Schur and Newton implementation of RQI and Rayleigh Chebyshev algorithms for real tensor pair of 
$T(I, X^{[m-1]}) - \lambda X = 0$.

Also compare with $O-NCM$, which is Newton-RQI, but we do a bit of code optimization.

We found it is much faster to compute the orthognal complement using the Householder transform directly instead of using the nullspace function.

In [50]:
def test_eigen_tensor(k, m, max_err, max_itr, n_test):   
    from time import process_time
    import pandas as pd
    A = ut.generate_symmetric_tensor(k, m)

    o_ncm_cnt = np.full(n_test, fill_value=np.nan)
    schur_cnt = np.full(n_test, fill_value=np.nan)
    newton_cnt = np.full(n_test, fill_value=np.nan)
    schur_cheb_cnt = np.full(n_test, fill_value=np.nan)

    o_ncm_err = np.full(n_test, fill_value=np.nan)
    schur_err = np.full(n_test, fill_value=np.nan)
    newton_err = np.full(n_test, fill_value=np.nan)
    schur_cheb_err = np.full(n_test, fill_value=np.nan)

    o_ncm_lbd = np.full(n_test, fill_value=np.nan)
    schur_lbd = np.full(n_test, fill_value=np.nan)
    newton_lbd = np.full(n_test, fill_value=np.nan)
    schur_cheb_lbd = np.full(n_test, fill_value=np.nan)

    o_ncm_time = np.full(n_test, fill_value=np.nan)
    schur_time = np.full(n_test, fill_value=np.nan)
    newton_time = np.full(n_test, fill_value=np.nan)
    schur_cheb_time = np.full(n_test, fill_value=np.nan)

    for jj in range(n_test):
        x0 = np.random.randn(k)
        x0 = x0 / np.linalg.norm(x0)

        # do orthogonal
        t_start = process_time()
        o_x, o_lbd, o_ctr, converge = uz.orthogonal_newton_correction_method(
            A, max_itr, max_err, x_init=x0)
        t_end = process_time()
        if converge:
            o_ncm_cnt[jj] = o_ctr
            o_ncm_lbd[jj] = o_lbd
            o_ncm_err[jj] = np.linalg.norm(
                ut.tv_mode_product(
                    A, o_x, m-1) - o_lbd * o_x)
            o_ncm_time[jj] = t_end - t_start

        # do schur_form_rayleigh
        t_start = process_time()
        # s_x, s_lbd, ctr, converge = schur_form_rayleigh_chebyshev_linear(
        # A, max_itr, max_err, x_init=x0, do_chebyshev=True)
        s_x, s_lbd, ctr, converge, err = uz.schur_form_rayleigh_chebyshev(
            A, max_itr, max_err, x_init=x0, do_chebyshev=False)

        t_end = process_time()
        if converge:
            schur_cnt[jj] = ctr
            schur_lbd[jj] = s_lbd
            schur_err[jj] = np.linalg.norm(
                ut.tv_mode_product(A, s_x, m-1) - s_lbd * s_x)
            schur_time[jj] = t_end - t_start

        # now do rayleigh
        t_start = process_time()
        ntn_x, ntn_lbd, ctr, converge, err = uz.newton_form_rayleigh_chebyshev(
            A, max_itr, max_err, x_init=x0, do_chebyshev=False)
        t_end = process_time()
        if converge:
            newton_time[jj] = t_end - t_start
            newton_cnt[jj] = ctr
            newton_lbd[jj] = ntn_lbd
            newton_err[jj] = np.linalg.norm(
                ut.tv_mode_product(
                    A, ntn_x, m-1) - ntn_lbd * ntn_x)

        # now do rayleigh chebyshev
        t_start = process_time()        
        sch_x, sch_lbd, ctr, converge, err = uz.schur_form_rayleigh_chebyshev(
              A, max_itr, max_err, x_init=x0, do_chebyshev=True)

        t_end = process_time()
        if converge:
            schur_cheb_time[jj] = t_end - t_start
            schur_cheb_cnt[jj] = ctr
            schur_cheb_lbd[jj] = sch_lbd
            schur_cheb_err[jj] = np.linalg.norm(
                ut.tv_mode_product(A, sch_x, m-1) - sch_lbd * sch_x)
            schur_cheb_time[jj] = t_end - t_start

    summ = pd.DataFrame(
        {
            'o_ncm_iter': o_ncm_cnt,
            'schur_iter': schur_cnt,
            'newton_iter': newton_cnt, 'schur_cheb_iter': schur_cheb_cnt,
            'o_ncm_err': o_ncm_err,
            'schur_err': schur_err,
            'newton_err': newton_err, 'schur_cheb_err': schur_cheb_err,
            'o_ncm_lbd': o_ncm_lbd,
            'schur_lbd': schur_lbd,
            'newton_lbd': newton_lbd,
            'schur_cheb_lbd': schur_cheb_lbd,
            'o_ncm_time': o_ncm_time,
            'schur_time': schur_time,
            'newton_time': newton_time,
            'schur_cheb_time': schur_cheb_time
        },
        columns=['o_ncm_iter', 'o_ncm_lbd', 'o_ncm_err', 'o_ncm_time',
                 'schur_iter', 'schur_lbd', 'schur_err', 'schur_time',
                 'newton_iter', 'newton_lbd', 'newton_err', 'newton_time',
                 'schur_cheb_iter', 'schur_cheb_lbd', 'schur_cheb_err',
                 'schur_cheb_time'])
    return summ        

In [51]:
np.random.seed(0)
k = 6
m = 3
max_err = 1e-10
max_itr = 200
n_test = 1000

summ = test_eigen_tensor(k, m, max_err, max_itr, n_test)
# summ[['o_ncm_time', 'schur_time', 'newton_time', 'newton_cheb_time']].describe())
# display(HTML(summ.describe().to_html()))
display(HTML(summ[[a for a in summ.columns if 'time' in a]].describe().to_html()))
display(HTML(summ[[a for a in summ.columns if 'iter' in a]].describe().to_html()))
display(HTML(summ[[a for a in summ.columns if 'lbd' in a]].describe().to_html()))
display(HTML(summ[[a for a in summ.columns if 'err' in a]].describe().to_html()))

,o_ncm_time,schur_time,newton_time,schur_cheb_time
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.014734,0.008142,0.005134,0.010171
std,0.011159,0.005823,0.004597,0.005350
min,0.003692,0.001294,0.001246,0.002464
25%,0.007841,0.003982,0.002480,0.006928
50%,0.011116,0.006955,0.003436,0.009044
75%,0.017415,0.010062,0.005530,0.012184
max,0.088810,0.051595,0.032517,0.037964


,o_ncm_iter,schur_iter,newton_iter,schur_cheb_iter
count,1000.0000,1000.000000,1000.000000,1000.000000
mean,13.7230,13.783000,13.702000,11.483000
std,10.1016,10.374369,9.989744,7.211915
min,5.0000,5.000000,5.000000,4.000000
25%,8.0000,8.000000,8.000000,7.000000
50%,10.0000,10.000000,10.000000,9.000000
75%,16.0000,16.000000,16.000000,14.000000
max,87.0000,83.000000,75.000000,55.000000


,o_ncm_lbd,schur_lbd,newton_lbd,schur_cheb_lbd
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.000939,-0.019187,-0.013954,0.001127
std,1.657999,1.579579,1.618327,1.472290
min,-8.041008,-8.041008,-8.041008,-8.041008
25%,-0.710710,-0.710710,-0.710710,-0.710710
50%,-0.000731,-0.000731,-0.000731,-0.000731
75%,0.710710,0.710710,0.710710,0.710710
max,8.041008,8.041008,8.041008,8.041008


,o_ncm_err,schur_err,newton_err,schur_cheb_err
count,1.000000e+03,1.000000e+03,1.000000e+03,1.000000e+03
mean,2.636017e-16,3.667934e-16,2.630650e-16,3.500143e-16
std,3.065191e-16,4.555482e-16,3.440989e-16,4.232397e-16
min,2.715272e-17,4.330081e-17,3.141718e-17,4.166949e-17
25%,1.405420e-16,1.888858e-16,1.401373e-16,1.862870e-16
50%,2.000851e-16,2.611270e-16,1.934700e-16,2.695539e-16
75%,2.857822e-16,3.863391e-16,2.733607e-16,3.816559e-16
max,3.044522e-15,4.094300e-15,3.293454e-15,4.925188e-15
